# Toronto Classification

**Jacob Bills**  
Data Systems Engineer

In [2]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import random # library for random number generation
import io
from sklearn.cluster import KMeans

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images 
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

#!conda install -c conda-forge geocoder --yes
import geocoder
print("Finished installing dependencies")

Finished installing dependencies


# Section 1- Parse Data from Wikipedia

In [5]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.replace("Not assigned", np.nan, inplace = True)
df.dropna(subset=["Borough"],axis=0,inplace=True)
df.loc[df['Neighbourhood'].isnull(),'Neighbourhood'] = df['Borough']
df=df.groupby(['Postcode','Borough']).Neighbourhood.agg([('count', 'count'), ('Neighbourhood', ','.join)]).reset_index()
df=df.drop(['count'],axis=1)
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Section 2 - Get Lattitude / Longitude

In [6]:
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
locdata=pd.read_csv(io.StringIO(s.decode('utf-8')))
locdata=locdata.rename(columns={"Postal Code": "Postcode"})
locdata.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_norm=df.set_index('Postcode').join(locdata.set_index('Postcode')).reset_index()
df_norm.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Section 3 - Clustering Toronto

## Display a map of Toronto area and plot points gather from section #2

In [49]:
address = 'Downtown Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6563221, -79.3809161.


In [50]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_norm['Latitude'], df_norm['Longitude'], df_norm['Borough'], df_norm['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Here we will build a classification model based on the most popular venues in downtown toronto borough. 

In [78]:
dt=df_norm[df_norm['Borough'] == 'Downtown Toronto']
#dt=df_norm.loc[(df_norm['Borough'] == 'Downtown Toronto')  | (df_norm['Borough'] == 'West Toronto')  | (df_norm['Borough'] == 'East Toronto')| (df_norm['Borough'] == 'East York')  | (df_norm['Borough'] == 'York'), :]
dt.head(5)
print(dt.shape)

(19, 5)


In [80]:
# create map of downtown toronto using latitude and longitude values
map_dttoronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(dt['Latitude'], dt['Longitude'], dt['Borough'], dt['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dttoronto)  
    
map_dttoronto

In [ ]:
# The hidden cell below must declare 3 variables for Foursquare API
# CLIENT_ID = '' # your Foursquare ID
# CLIENT_SECRET = '' # your 

In [58]:
# The code was removed by Watson Studio for sharing.

In [81]:
LIMIT = 100
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Get top 100 venues for each neighbourhood in downtown (dt) dataset up to 500 meters from latitude and longitude

In [82]:
dt_venues = getNearbyVenues(names=dt['Neighbourhood'],
                                   latitudes=dt['Latitude'],
                                   longitudes=dt['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Queen's Park


In [83]:
dt_venues.head(5)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown,St. James Town",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant


### Get total unique categories across all venues from each neighbourhood

In [84]:
print('There are {} uniques categories.'.format(len(dt_venues['Venue Category'].unique())))

There are 200 uniques categories.


### Use one-hot encoding to get a mean distribution of venues for each neighbourhood across the whole data set

In [85]:
# one hot encoding
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighbourhood'] = dt_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_grouped = dt_onehot.groupby('Neighbourhood').mean().reset_index()
dt_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,...,0.0,0.0,0.0,0.020000,0.0,0.0,0.010000,0.0,0.01,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.018182,0.0,0.0,0.000000,0.0,0.00,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000
3,"Cabbagetown,St. James Town",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000
4,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.0,...,0.0,0.0,0.0,0.011765,0.0,0.0,0.011765,0.0,0.00,0.011765


In [86]:
dt_grouped.shape

(19, 201)

### Create dataframe giving nth rank to venues from highest to lowest up to 10

In [87]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhood_venues_sorted = pd.DataFrame(columns=columns)
neighbourhood_venues_sorted['Neighbourhood'] = dt_grouped['Neighbourhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighbourhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighbourhood_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Cosmetics Shop,Bakery,Restaurant,Asian Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Bakery,Steakhouse,Cocktail Bar,Café,French Restaurant
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Plane,Sculpture Garden,Boutique,Boat or Ferry
3,"Cabbagetown,St. James Town",Coffee Shop,Park,Restaurant,Pub,Café,Bakery,Pizza Place,Chinese Restaurant,Italian Restaurant,Jewelry Store
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Juice Bar,Burger Joint,Japanese Restaurant,Ice Cream Shop,Chinese Restaurant,Bar


## Cluster neighbourhoods

In [88]:
kclusters = 5

dt_grouped_clustering = dt_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 3, 2, 2, 2, 4, 2, 2, 2], dtype=int32)

### add labeling to clusters

In [89]:
# add clustering labels
neighbourhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
dt_merged = dt
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dt_merged = dt_merged.join(neighbourhood_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
dt_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
51,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,2,Coffee Shop,Park,Restaurant,Pub,Café,Bakery,Pizza Place,Chinese Restaurant,Italian Restaurant,Jewelry Store
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Fast Food Restaurant,Gym,Hotel,Café,Gastropub
53,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Pub,Café,Bakery,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market,Chocolate Shop
54,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Cosmetics Shop,Diner,Restaurant,Bookstore,Pizza Place,Fast Food Restaurant


In [91]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighbourhood'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [92]:
dt_merged.loc[dt_merged['Cluster Labels'] == 0, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Downtown Toronto,0,Coffee Shop,Park,Pub,Café,Bakery,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market,Chocolate Shop
85,Downtown Toronto,0,Coffee Shop,Park,Gym,College Auditorium,Seafood Restaurant,Sandwich Place,Salad Place,Burger Joint,Portuguese Restaurant,Burrito Place


In [96]:
dt_merged.loc[dt_merged['Cluster Labels'] == 1, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,1,Park,Playground,Trail,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [97]:
dt_merged.loc[dt_merged['Cluster Labels'] == 2, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Downtown Toronto,2,Coffee Shop,Park,Restaurant,Pub,Café,Bakery,Pizza Place,Chinese Restaurant,Italian Restaurant,Jewelry Store
52,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Fast Food Restaurant,Gym,Hotel,Café,Gastropub
54,Downtown Toronto,2,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Cosmetics Shop,Diner,Restaurant,Bookstore,Pizza Place,Fast Food Restaurant
55,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Cocktail Bar,Cosmetics Shop,Bakery,Breakfast Spot,Italian Restaurant,Hotel,American Restaurant
56,Downtown Toronto,2,Coffee Shop,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Bakery,Steakhouse,Cocktail Bar,Café,French Restaurant
57,Downtown Toronto,2,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Juice Bar,Burger Joint,Japanese Restaurant,Ice Cream Shop,Chinese Restaurant,Bar
58,Downtown Toronto,2,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Cosmetics Shop,Bakery,Restaurant,Asian Restaurant,Burger Joint
59,Downtown Toronto,2,Coffee Shop,Aquarium,Café,Italian Restaurant,Hotel,Scenic Lookout,Brewery,Restaurant,Fried Chicken Joint,Baseball Stadium
60,Downtown Toronto,2,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Steakhouse,Deli / Bodega,Bar,Seafood Restaurant,American Restaurant
61,Downtown Toronto,2,Coffee Shop,Café,Hotel,Restaurant,Gym,Deli / Bodega,Gastropub,Italian Restaurant,Seafood Restaurant,American Restaurant


In [98]:
dt_merged.loc[dt_merged['Cluster Labels'] == 3, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Plane,Sculpture Garden,Boutique,Boat or Ferry


In [99]:
dt_merged.loc[dt_merged['Cluster Labels'] == 4, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Downtown Toronto,4,Grocery Store,Café,Park,Coffee Shop,Diner,Gas Station,Nightclub,Baby Store,Restaurant,Italian Restaurant


## Coffee shops are closer to cafes and restaurants than they are closer to parks and are the most popular venue type for downtown toronto